In [1]:
# --- Imports ---
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [2]:
from PIL import Image
import os

def convert_to_png(input_path, output_path=None):
    
    try:
        img = Image.open(input_path).convert("RGBA")
    except Exception as e:
        print(f"failed o open: {e}")
        return None

    if output_path is None:
        base, _ = os.path.splitext(input_path)
        output_path = base + ".png"

    try:
        img.save(output_path, format="PNG")
        print(f"saveed PNG：{output_path}")
        return output_path
    except Exception as e:
        print(f"error on saving PNG：{e}")
        return None
    


In [3]:
# --- Convert .pgm to cleaned .png (white stays, others become black) ---
def convert_clean_img_to_png(img, png_path,white_threshold=240):
    img_array = np.array(img.convert('L')) 
    img_array = np.array(img)

    # Set all non-white pixels to black
    cleaned = np.where(img_array >= white_threshold, 255, 0).astype(np.uint8)
    cleaned_img = Image.fromarray(cleaned)
    cleaned_img.save(png_path)


In [4]:
def pad_image(png_path, padded_path, cell_size=5):
    img = Image.open(png_path).convert('L')
    img_array = np.array(img)

    h, w = img_array.shape
    pad_h = (cell_size - (h % cell_size)) % cell_size
    pad_w = (cell_size - (w % cell_size)) % cell_size

    # Pad with black pixels (0) at bottom and right
    padded_array = np.pad(
        img_array,
        ((0, pad_h), (0, pad_w)),
        mode='constant',
        constant_values=0
    )

    padded_img = Image.fromarray(padded_array)
    padded_img.save(padded_path)



In [5]:


# --- Convert cleaned PNG to 5x5 grid ---
def png_to_grid(padded_path, cell_size=5, output_dir="grid_txt"):
    # Prepare output directory
    os.makedirs(output_dir, exist_ok=True)

    # Derive filename from input path
    base_name = os.path.splitext(os.path.basename(padded_path))[0]
    output_txt = os.path.join(output_dir, f"{base_name}Matrix.txt")

    # Load and convert image
    img = Image.open(padded_path).convert('L')
    img_array = np.array(img)
    h, w = img_array.shape

    new_h = h // cell_size
    new_w = w // cell_size
    img_array = img_array[:new_h * cell_size, :new_w * cell_size]

    # Convert to grid
    reshaped = img_array.reshape(new_h, cell_size, new_w, cell_size)
    cell_min = reshaped.min(axis=(1, 3))
    grid = (cell_min < 255).astype(int)

    # Remove outer rows/columns that are all 1s
    row_mask = ~(grid == 1).all(axis=1)
    col_mask = ~(grid == 1).all(axis=0)
    trimmed_grid = grid[np.ix_(row_mask, col_mask)]
    grid = np.pad(trimmed_grid, pad_width=1, mode='constant', constant_values=1)

    # Save to txt
    np.savetxt(output_txt, grid, fmt="%d", delimiter=" ")
    print(f"📄 Grid saved to {output_txt}")
    return grid


In [6]:
# --- Visualize grid ---
def show_grid(grid):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(grid, cmap='gray_r', interpolation='nearest')

    # Draw green grid lines
    rows, cols = grid.shape
    for x in range(cols + 1):
        ax.axvline(x - 0.5, color='green', linewidth=0.5)
    for y in range(rows + 1):
        ax.axhline(y - 0.5, color='green', linewidth=0.5)

    ax.set_title("5x5 Grid Map with Gridlines")
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()


In [7]:
def save_grid_as_png(grid, output_path):
    img_array = np.where(grid == 0, 255, 0).astype(np.uint8)
    img = Image.fromarray(img_array, mode='L')
    img.save(output_path)


In [8]:
import os
from PIL import Image

def smart_batch_convert(raw_dir="raw_images", out_dir="converted_images"):
    """
    Converts image files in `raw_dir` to PNG format and saves them in `out_dir`.
    Only processes files that haven't been converted yet.
    Automatically creates folders if they don't exist.
    """
    # Step 1: Ensure directories exist
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)

    # Supported image formats
    valid_exts = ['.jpg', '.jpeg', '.png', '.bmp', '.pgm', '.gif', '.tiff', '.webp', '.heic', '.jfif', '.tga']

    # Stats
    converted = 0
    skipped = 0

    # Step 2: Scan input directory
    for filename in os.listdir(raw_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() not in valid_exts:
            continue

        input_path = os.path.join(raw_dir, filename)
        output_path = os.path.join(out_dir, name + ".png")

        if os.path.exists(output_path):
            print(f"✅ Skipped (already exists): {name}.png")
            skipped += 1
            continue

        try:
            img = Image.open(input_path).convert("RGBA")
            img.save(output_path, format="PNG")
            base_name = os.path.splitext(os.path.basename(input_path))[0]
            cleaned_img = os.path.join(out_dir, f"{base_name}_cleaned.png")
            padded_img = os.path.join(out_dir, f"{base_name}_padded.png")
            convert_clean_img_to_png(img, cleaned_img)
            pad_image(cleaned_img, padded_img)
            grid = png_to_grid(padded_img, cell_size=5)
            # show_grid(grid)

            save_grid_as_png(grid, output_path)
            os.remove(cleaned_img)
            os.remove(padded_img)
            print(f"🎯 Converted: {filename} → {name}.png")
            converted += 1
        except Exception as e:
            print(f"❌ Failed to convert {filename}: {e}")

    print(f"\n✅ Done! {converted} converted, {skipped} skipped.")


In [9]:
# # --- Run all steps ---
smart_batch_convert()
# png_path = 'map_cleaned.png'

# convert_to_png(img_path)
# convert_clean_img_to_png(img_path, png_path)
# png_path = 'map_cleaned.png'
# padded_path = 'map_padded.png'
# pad_image(png_path, padded_path)
# grid = png_to_grid(png_path, cell_size=5)
# show_grid(grid)

# save_grid_as_png(grid, 'final_grid.png')


📄 Grid saved to grid_txt\map_paddedMatrix.txt
🎯 Converted: map.pgm → map.png
📄 Grid saved to grid_txt\OIP_paddedMatrix.txt
🎯 Converted: OIP.jfif → OIP.png

✅ Done! 2 converted, 0 skipped.
